In [ ]:
#dictionary function tester: 

#change to "wrapped" function style? 
see: python social media analytics pg 297.5 / 535
    
    
    
    #see learn python programming + fabrizio R (PG 245 / 1052)
    using: for data in zip(a,b,c):
            a,b,c = data
            
            #zip + multiple iteration style 

In [4]:
#system packages

import sys
import warnings
import os 
import traceback #obs? 
if not sys.warnoptions:
    warnings.filterwarnings("once")  

In [5]:
#base packages:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats, integrate, optimize
import math
import datetime
from datetime import date, timedelta

#optional packages

from statsmodels.tsa.base.datetools import dates_from_str
from IPython.display import display
pd.options.display.max_columns = 60  #allow DF.head to show all columns in notebook
from see import see
from tabulate import tabulate 
#from io import StringIO

In [6]:
#packages for the econometrics / models

from statsmodels.tsa.vector_ar import vecm
from statsmodels.tsa.stattools import adfuller
from arch.unitroot import DFGLS, ADF, KPSS, PhillipsPerron, ZivotAndrews
from arch.unitroot.cointegration import engle_granger, phillips_ouliaris
import statsmodels.formula.api as smf  #VAR package contained within 
import statsmodels.tsa.api as smt

#import the functionality for detecting mathematical errors (E.G. types of linear algebra issues etc.)
from statsmodels.tools.sm_exceptions import ValueWarning
from arch.utility.exceptions import (
    InfeasibleTestException,
    InvalidLengthWarning,
    invalid_length_doc)
warnings.filterwarnings("once", category = ValueWarning)

C:\Users\JasonSpano\anaconda3\lib\site-packages\patsy\constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [7]:
#pickle file: 
#sorted_alldata_df_final = pd.read_pickle("./sorted_alldata_df_final.pkl")

sorted_alldata_df_final = pd.read_pickle(r"C:\Users\JasonSpano\OneDrive\Uva Msc 19-20 Subjects\Research Assistant Docs\CDS_project_-_first_set_of_tasks\Part 2_jan20./sorted_alldata_df_final.pkl")

In [ ]:
#with ADF results: to be updated to VECM outcome (cointegration) eventually?
#sorted_alldata_df_final_adf_results = pd.read_pickle("./sorted_alldata_df_final_adf_results.pkl")
 

In [9]:
gvkey_rand = sorted_alldata_df_final.loc[sorted_alldata_df_final.gvkey==1045]

In [16]:
class regression_packages:
    def __init__(self, autolag):
        self.autolag = str(bic)
        
        
    
    
    def adf_base(self):
        cds = self.market_cds_spread
        pecds = self.pecds
        data_df = np.column_stack((cds, pecds))
        
        try:
            model = smt.VAR(data_df)  
            bic_selected_lag = model.select_order(maxlags=10).bic 
        except (np.linalg.LinAlgError, ValueError):
            print("Instance Error Occured")
            pass
        
        return bic_selected_lag
    
    
        
        
        

In [17]:
regression_packages.adf_base(gvkey_rand)

10

In [26]:
#simple tester from lists+append >> dictionary: 

gvkey_list = []                       #holds the GVKEY
sector_list = []                      #append the sector for each GVKEY (just saves having to re-merge on again later)
optimal_lag_list = []                 #holds the optimal number of lags selected by the BIC
gvkey_sector_lag_dict = {}
problematic_gvkey = []
linalg_errors = 0; value_errors = 0

for i, val in enumerate(sorted_alldata_df_final.groupby('gvkey')):
    gvkey = val[0]
    data = val[1]                        #assign the remainder of the data to be callable from data.(variable)
    gvkey_list.append(gvkey)             #append the GVKEY to a list, which keeps the data stored under the appropriate key
    
    sector_string = data.sector
    sector_list.append(sector_string.iloc[0])      #append the respective sector for that GVKEY
    
    #dicts
    gvkey_sector_lag_dict[(i)] = gvkey
    #{gvkey_sector_lag_dict: gvkey for i in gvkey}
    gvkey_sector_lag_dict[(i)] = data.sector
    
    #define the appropriate series for the ADF test: 
    cds = data.market_cds_spread         
    pecds = data.pecds
    
    data_df = np.column_stack((cds, pecds)) 
    
    #the VAR model construction is necessary for computing the BIC
    model = smt.VAR(data_df)  
    
    #compute the optimised BIC value from [model] which holds the vector autoregression model per GVKEY
    try:
        #for daily data, a maximum lag length is imposed (5 trading days, roughly equivalent to one trading week)
        bic_selected_lag = model.select_order(maxlags=10).bic 
        optimal_lag_list.append(bic_selected_lag)                  #store the optimised BIC result per GVKEY   
        
        
        
    except np.linalg.LinAlgError as lae:
        problematic_gvkey.append(gvkey)                                 #append problem GVKEY to seperate list
        print("LinAlgError: Row: {}, Reason : {} ".format(i, lae), '\n', "GVKEY: ({})".format(gvkey), 
              "SECTOR: ", sector_string.iloc[0])   
        optimal_lag_list.append(np.nan)
        linalg_errors += 1
        
        
    except (ValueError) as ve:
        problematic_gvkey.append(gvkey)
        print("ValueError: Row: {}, Reason : {} ".format(i, ve), '\n', "GVKEY: ({})".format(gvkey), 
              "SECTOR:", sector_string.iloc[0])
        
        optimal_lag_list.append(np.nan)    
        value_errors += 1

        
#the final DF constructor; the .T is to transpose to a more visually-friendly vertically orientated DF
df_stat_cdsbond_dict_results = pd.DataFrame(data=(gvkey_list, sector_list, optimal_lag_list)).T

#rename the columns for better readability
df_stat_cdsbond_dict_results.columns = ['gvkey', 'sector',  'bic_lag'] 

print('\n', " Linear Algebra Errors #", linalg_errors, '\n', " Value Errors #", value_errors, '\n', " Total Errors #:", 
     (linalg_errors + value_errors))   

ValueError: Row: 105, Reason : x contains a constant. Adding a constant with trend='c' is not allowed.  
 GVKEY: (5671) SECTOR: Consumer Cyclical
LinAlgError: Row: 109, Reason : 1-th leading minor of the array is not positive definite  
 GVKEY: (5862) SECTOR:  Materials
ValueError: Row: 123, Reason : x contains a constant. Adding a constant with trend='c' is not allowed.  
 GVKEY: (6653) SECTOR: Financials
ValueError: Row: 158, Reason : x contains a constant. Adding a constant with trend='c' is not allowed.  
 GVKEY: (7991) SECTOR: Capital Goods
ValueError: Row: 179, Reason : x contains a constant. Adding a constant with trend='c' is not allowed.  
 GVKEY: (8551) SECTOR: Consumer Cyclical
ValueError: Row: 205, Reason : x contains a constant. Adding a constant with trend='c' is not allowed.  
 GVKEY: (10187) SECTOR: Financials
ValueError: Row: 207, Reason : x contains a constant. Adding a constant with trend='c' is not allowed.  
 GVKEY: (10247) SECTOR: Consumer Staples
LinAlgError: Row

In [27]:
gvkey_sector_lag_dict

{0: 0      Transporation
 1      Transporation
 2      Transporation
 3      Transporation
 4      Transporation
            ...      
 541    Transporation
 542    Transporation
 543    Transporation
 544    Transporation
 545    Transporation
 Name: sector, Length: 546, dtype: object,
 1: 546     Health Care
 547     Health Care
 548     Health Care
 549     Health Care
 550     Health Care
            ...     
 1161    Health Care
 1162    Health Care
 1163    Health Care
 1164    Health Care
 1165    Health Care
 Name: sector, Length: 620, dtype: object,
 2: 1166    Technology
 1167    Technology
 1168    Technology
 1169    Technology
 1170    Technology
            ...    
 3046    Technology
 3047    Technology
 3048    Technology
 3049    Technology
 3050    Technology
 Name: sector, Length: 1885, dtype: object,
 3: 3051    Materials
 3052    Materials
 3053    Materials
 3054    Materials
 3055    Materials
           ...    
 4517    Materials
 4518    Materials
 4519    Mate

In [19]:
gvkey_dict = pd.DataFrame((gvkey_sector_lag_dict.items()),columns=['GVKEY','SECTOR'])

In [20]:
gvkey_dict

,GVKEY,SECTOR
0,0,0 Transporation 1 Transporation 2 ...
1,1,546 Health Care 547 Health Care 548 ...
2,2,1166 Technology 1167 Technology 1168 ...
3,3,3051 Materials 3052 Materials 3053 Ma...
4,4,4522 Capital Goods 4523 Capital Goods 45...
...,...,...
386,386,454134 Consumer Cyclical 454135 Consumer...
387,387,454399 Communication Services 454400 Com...
388,388,456491 Materials 456492 Materials 456493...
389,389,458251 Capital Goods 458252 Capital Good...


In [ ]:
df_stat_cdsbond_dict_results

In [ ]:
#functioning with lists: mini version 

In [7]:
#simple tester from lists+append >> dictionary: 

gvkey_list = []                       #holds the GVKEY
sector_list = []                      #append the sector for each GVKEY (just saves having to re-merge on again later)
optimal_lag_list = []                 #holds the optimal number of lags selected by the BIC
problematic_gvkey = []
linalg_errors = 0; value_errors = 0

for i, val in enumerate(sorted_alldata_df_final.groupby('gvkey')):
    gvkey = val[0]
    data = val[1]                        #assign the remainder of the data to be callable from data.(variable)
    gvkey_list.append(gvkey)             #append the GVKEY to a list, which keeps the data stored under the appropriate key
    sector_string = data.sector
    sector_list.append(sector_string.iloc[0])      #append the respective sector for that GVKEY
    
    #define the appropriate series for the ADF test: 
    cds = data.market_cds_spread         
    pecds = data.pecds
    
    data_df = np.column_stack((cds, pecds)) 
    
    #the VAR model construction is necessary for computing the BIC
    model = smt.VAR(data_df)  
    
    #compute the optimised BIC value from [model] which holds the vector autoregression model per GVKEY
    try:
        #for daily data, a maximum lag length is imposed (5 trading days, roughly equivalent to one trading week)
        bic_selected_lag = model.select_order(maxlags=10).bic 
        optimal_lag_list.append(bic_selected_lag)                  #store the optimised BIC result per GVKEY   
        
        
        
    except np.linalg.LinAlgError as lae:
        problematic_gvkey.append(gvkey)                                 #append problem GVKEY to seperate list
        print("LinAlgError: Row: {}, Reason : {} ".format(i, lae), '\n', "GVKEY: ({})".format(gvkey), 
              "SECTOR: ", sector_string.iloc[0])   
        optimal_lag_list.append(np.nan)
        linalg_errors += 1
        
        
    except (ValueError) as ve:
        problematic_gvkey.append(gvkey)
        print("ValueError: Row: {}, Reason : {} ".format(i, ve), '\n', "GVKEY: ({})".format(gvkey), 
              "SECTOR:", sector_string.iloc[0])
        
        optimal_lag_list.append(np.nan)    
        value_errors += 1

        
#the final DF constructor; the .T is to transpose to a more visually-friendly vertically orientated DF
df_stat_cdsbond_dict_results = pd.DataFrame(data=(gvkey_list, sector_list, optimal_lag_list)).T

#rename the columns for better readability
df_stat_cdsbond_dict_results.columns = ['gvkey', 'sector',  'bic_lag'] 

print('\n', " Linear Algebra Errors #", linalg_errors, '\n', " Value Errors #", value_errors, '\n', " Total Errors #:", 
     (linalg_errors + value_errors))        

ValueError: Row: 105, Reason : x contains a constant. Adding a constant with trend='c' is not allowed.  
 GVKEY: (5671) SECTOR: Consumer Cyclical
LinAlgError: Row: 109, Reason : 1-th leading minor of the array is not positive definite  
 GVKEY: (5862) SECTOR:  Materials
ValueError: Row: 123, Reason : x contains a constant. Adding a constant with trend='c' is not allowed.  
 GVKEY: (6653) SECTOR: Financials
ValueError: Row: 158, Reason : x contains a constant. Adding a constant with trend='c' is not allowed.  
 GVKEY: (7991) SECTOR: Capital Goods
ValueError: Row: 179, Reason : x contains a constant. Adding a constant with trend='c' is not allowed.  
 GVKEY: (8551) SECTOR: Consumer Cyclical
ValueError: Row: 205, Reason : x contains a constant. Adding a constant with trend='c' is not allowed.  
 GVKEY: (10187) SECTOR: Financials
ValueError: Row: 207, Reason : x contains a constant. Adding a constant with trend='c' is not allowed.  
 GVKEY: (10247) SECTOR: Consumer Staples
LinAlgError: Row

In [8]:
df_stat_cdsbond_dict_results

,gvkey,sector,bic_lag
0,1045,Transporation,10
1,1078,Health Care,1
2,1161,Technology,2
3,1209,Materials,1
4,1300,Capital Goods,2
...,...,...,...
386,175263,Consumer Cyclical,1
387,176404,Communication Services,2
388,176760,Materials,2
389,179657,Capital Goods,NaN


In [ ]:
#OG: 

#initialise a few lists to append data to from each GVKEY's loop

gvkey_list = []                       #holds the GVKEY
sector_list = []                      #append the sector for each GVKEY (just saves having to re-merge on again later)
optimal_lag_list = []                 #holds the optimal number of lags selected by the BIC
first_date_obs = []                   #holds the first date observation per GVKEY
last_date_obs = []                    #holds the last date observation per GVKEY
time_horizon = []                     #holds the difference between the first and last date observations

stationarity_ts_cds = []              #the test statistic for the CDS series
stationarity_ts_bond = []             #the test statistic for the BOND series
stationarity_ts_vix = []

stationarity_pvalue_cds = []          #the p-value for the CDS series
stationarity_pvalue_bond = []         #the p-value for the BOND series
stationarity_pvalue_vix = []

stationarity_test_res_cds = []        #a list which will hold the significance level (*, **, ***) for publication tables
stationarity_test_res_bond = []
stationarity_test_res_vix = []

stat_test_joint = []                  #will hold the outcome for the joint unit-root test outcome
problematic_gvkey = []                #hold those GVKEYS with either linear algebra or value errors: 
linalg_errors = 0; value_errors = 0

#THE ADF CRITICAL VALUES - highlighted/calculated above:
one_percent_cv = -3.4347287139577753   
five_percent_cv = -2.8634739225299133 
ten_percent_cv = -2.5677995973755325
adf_automated_lag_cds = []; adf_automated_lag_bond = []

#create loop to test for stationarity in each GVKEY's data group 
for i, val in enumerate(sorted_alldata_df_final.groupby('gvkey')):
    gvkey = val[0]
    data = val[1]                        #assign the remainder of the data to be callable from data.(variable)
    gvkey_list.append(gvkey)             #append the GVKEY to a list, which keeps the data stored under the appropriate key
    sector_string = data.sector
    sector_list.append(sector_string.iloc[0])      #append the respective sector for that GVKEY
    
    #compute the specifics of each GVKEY's date / time horizon
    first_date_obs.append(data.trade_date.min())
    last_date_obs.append(data.trade_date.max())
    time_difference = data.trade_date.max() - data.trade_date.min()
    time_horizon.append(time_difference.days)               #append just the .days aspect of the timedelta variable
    
    #define the appropriate series for the ADF test: 
    cds = data.market_cds_spread         
    pecds = data.pecds
    vix = data.cboe_vix
    
    #the VAR model package requires the data be structured in a particular way; np.column stack is the fastest method
    data = np.column_stack((cds, pecds)) 
    
    #the VAR model construction is necessary for computing the BIC
    model = smt.VAR(data)  

    #compute the optimised BIC value from [model] which holds the vector autoregression model per GVKEY
    try:
        #for daily data, a maximum lag length is imposed (5 trading days, roughly equivalent to one trading week)
        bic_selected_lag = model.select_order(maxlags=10).bic 
        optimal_lag_list.append(bic_selected_lag)                  #store the optimised BIC result per GVKEY
        
        #ADF test - plugging in the bic selected lag length to run a unique length ADF test per GVKEY, per instrument
        test_ADF_cds = ADF(cds, trend='c', max_lags=10, method='BIC') 
        adf_automated_lag_cds.append(test_ADF_cds.lags)
        
        test_ADF_bond = ADF(pecds, trend='c',  max_lags=10, method='BIC')  
        adf_automated_lag_bond.append(test_ADF_bond.lags)
        
        test_ADF_vix = ADF(vix, trend='c',  max_lags=10, method='BIC')
        #test_ADF_vix = ADF(vix, lags=bic_selected_lag)
        
        #store the test statistic & p-value for each instrument's ADF test result, for each GVKEY (rounded to 4 decimals)
        stationarity_ts_cds.append(np.round(test_ADF_cds.stat,4))    
        stationarity_ts_bond.append(np.round(test_ADF_bond.stat,4))     
        stationarity_ts_vix.append(np.round(test_ADF_vix.stat,4))
        
        stationarity_pvalue_cds.append(np.round(test_ADF_cds.pvalue,4))    
        stationarity_pvalue_bond.append(np.round(test_ADF_bond.pvalue,4))
        stationarity_pvalue_vix.append(np.round(test_ADF_vix.pvalue,4))
        
        ##OPTIONAL
        #this part reads the test stat per GVKEY / instrument and assigns the appropriate (*) per its significance level
        #E.G. > 99% significance is commonly denoted as [test_statistic]*** in publication regression tables
        
        #CDS ADF LATEX TABLE CONSTRUCTION
        if test_ADF_cds.stat < one_percent_cv:
            stationarity_test_res_cds.append("***")   
        elif test_ADF_cds.stat < five_percent_cv:
            stationarity_test_res_cds.append("**")    ## The 1% & 5% significance would reject H0
        elif test_ADF_cds.stat < ten_percent_cv:
            stationarity_test_res_cds.append("*")     ## The 10% and no level would fail to reject H0
        else: 
            stationarity_test_res_cds.append(" ")  

        #repeat for BONDS:
        if test_ADF_bond.stat < one_percent_cv:
            stationarity_test_res_bond.append("***")
        elif test_ADF_bond.stat < five_percent_cv:
            stationarity_test_res_bond.append("**")   ## The 1% & 5% significance would reject H0
        elif test_ADF_bond.stat < ten_percent_cv:
            stationarity_test_res_bond.append("*")    ## The 10% and no level would fail to reject H0
        else: 
            stationarity_test_res_bond.append(" ")
            
            
        #VIX ADF LATEX TABLE CONSTRUCTION
        if test_ADF_vix.stat < one_percent_cv:
            stationarity_test_res_vix.append("***")   
        elif test_ADF_vix.stat < five_percent_cv:
            stationarity_test_res_vix.append("**")    ## The 1% & 5% significance would reject H0
        elif test_ADF_vix.stat < ten_percent_cv:
            stationarity_test_res_vix.append("*")     ## The 10% and no level would fail to reject H0
        else: 
            stationarity_test_res_vix.append(" ")  
                    
        
        #ensure that both series are integrated of the same order: a jointly unit-root system is required for VECM models
        if (test_ADF_bond.stat > five_percent_cv and test_ADF_cds.stat > five_percent_cv):  
            #this is > as to "not reject" the H0 is unit root; while reject the null would imply a stationary series
            stat_test_joint.append("BOND/CDS Both Have Unit Root")
            #stat_boolean.append(int(1))
        elif (test_ADF_bond.stat > five_percent_cv and test_ADF_cds.stat < five_percent_cv):
            stat_test_joint.append("BOND ONLY HAS UR")
        elif (test_ADF_bond.stat < five_percent_cv and test_ADF_cds.stat > five_percent_cv):
            stat_test_joint.append("CDS ONLY HAS UR")            
        else: 
            stat_test_joint.append("Cannot Reject H0")
            #stat_boolean.append(int(2))
        
    # A linear-algebra error could be caused by market factors which have created highly erratic basis behaviour
    # This could be caused by a firm nearing bankruptcy where its basis deviates excessively far from par
    #accordingly, the leading minor of the array is not positive definite is akin to taking the square root of a negative number
    except np.linalg.LinAlgError as lae:
        problematic_gvkey.append(gvkey)                                 #append problem GVKEY to seperate list
        print("LinAlgError: Row: {}, Reason : {} ".format(i, lae), '\n', "GVKEY: ({})".format(gvkey), 
              "SECTOR: ", sector_string.iloc[0])
      
        #append items to maintain placement in main DF \\ 
        #where the errors occur, fill with NaN values instead
        optimal_lag_list.append(np.nan)
        adf_automated_lag_cds.append(np.nan)
        adf_automated_lag_bond.append(np.nan)
        stat_test_joint.append("LinAlg_Error")
        
        stationarity_test_res_cds.append("_LAE")
        stationarity_test_res_bond.append("_LAE")
        stationarity_test_res_vix.append("_LAE")
        
        stationarity_ts_cds.append(np.nan)
        stationarity_ts_bond.append(np.nan)
        stationarity_ts_vix.append(np.nan)
        
        stationarity_pvalue_cds.append(np.nan)
        stationarity_pvalue_bond.append(np.nan)
        stationarity_pvalue_vix.append(np.nan)
        
        linalg_errors += 1
        continue
    
    #some series may have encounted some errors with a constant, thus adding a constant to the equation is invalid
    #this occurs due to poor liquidity in some markets, resulting in long periods of near-identical price quotes
    except (ValueError) as ve:
        problematic_gvkey.append(gvkey)
        print("ValueError: Row: {}, Reason : {} ".format(i, ve), '\n', "GVKEY: ({})".format(gvkey), 
              "SECTOR:", sector_string.iloc[0])
        
        #append items to maintain placement in main DF \\ 
        #where the errors occur, fill with NaN values instead
        optimal_lag_list.append(np.nan)
        adf_automated_lag_cds.append(np.nan)
        adf_automated_lag_bond.append(np.nan)        
        stat_test_joint.append("Value_Error")
        
        stationarity_test_res_cds.append("_VE")
        stationarity_test_res_bond.append("_VE")
        stationarity_test_res_vix.append("_VE")        
        
        stationarity_ts_cds.append(np.nan)
        stationarity_ts_bond.append(np.nan)
        stationarity_ts_vix.append(np.nan)
        
        stationarity_pvalue_cds.append(np.nan)
        stationarity_pvalue_bond.append(np.nan)
        stationarity_pvalue_vix.append(np.nan) 
        
        value_errors += 1
        continue  
        
#this section combines the test-statistic itself with the appropriate (*, **, ***) into one cell for the final DF
cds_sig = ["{}{}".format(stationarity_ts_cds, stationarity_test_res_cds) for stationarity_ts_cds, stationarity_test_res_cds in zip(stationarity_ts_cds, stationarity_test_res_cds)]
bond_sig = ["{}{}".format(stationarity_ts_bond, stationarity_test_res_bond) for stationarity_ts_bond, stationarity_test_res_bond in zip(stationarity_ts_bond, stationarity_test_res_bond)]  
vix_sig = ["{}{}".format(stationarity_ts_vix, stationarity_test_res_vix) for stationarity_ts_vix, stationarity_test_res_vix in zip(stationarity_ts_vix, stationarity_test_res_vix)]        

#the final DF constructor; the .T is to transpose to a more visually-friendly vertically orientated DF
df_stat_cdsbond_results = pd.DataFrame(data=(gvkey_list, sector_list, 
                                             cds_sig, adf_automated_lag_cds, stationarity_pvalue_cds,
                                             bond_sig, adf_automated_lag_bond, stationarity_pvalue_bond,
                                             vix_sig, stationarity_pvalue_vix,
                                             stat_test_joint, optimal_lag_list,
                                            first_date_obs, last_date_obs, time_horizon)).T

#dont think the boolean is needed? can read col names fine? 

#rename the columns for better readability
df_stat_cdsbond_results.columns = ['gvkey', 'sector', 
                                   'CDS', 'CDS_ADF_LAGS','CDS_pval', 
                                   'Bond', 'BOND_ADF_LAGS','BOND_pval',
                                   'VIX', 'VIX_pval',
                                   'Joint_Test_Outcome', 'bic_lag',
                                  'date_start','date_end','length_days'] 

#change to tabulate? 
print('\n', " Linear Algebra Errors #", linalg_errors, '\n', " Value Errors #", value_errors, '\n', " Total Errors #:", 
     (linalg_errors + value_errors))

In [ ]:
#ORIGINAL ADF: 



#initialise a few lists to append data to from each GVKEY's loop

gvkey_list = []                       #holds the GVKEY
sector_list = []                      #append the sector for each GVKEY (just saves having to re-merge on again later)
optimal_lag_list = []                 #holds the optimal number of lags selected by the BIC
first_date_obs = []                   #holds the first date observation per GVKEY
last_date_obs = []                    #holds the last date observation per GVKEY
time_horizon = []                     #holds the difference between the first and last date observations

stationarity_ts_cds = []              #the test statistic for the CDS series
stationarity_ts_bond = []             #the test statistic for the BOND series
stationarity_ts_vix = []

stationarity_pvalue_cds = []          #the p-value for the CDS series
stationarity_pvalue_bond = []         #the p-value for the BOND series
stationarity_pvalue_vix = []

stationarity_test_res_cds = []        #a list which will hold the significance level (*, **, ***) for publication tables
stationarity_test_res_bond = []
stationarity_test_res_vix = []

stat_test_joint = []                  #will hold the outcome for the joint unit-root test outcome
problematic_gvkey = []                #hold those GVKEYS with either linear algebra or value errors: 
linalg_errors = 0; value_errors = 0

#THE ADF CRITICAL VALUES - highlighted/calculated above:
one_percent_cv = -3.4347287139577753   
five_percent_cv = -2.8634739225299133 
ten_percent_cv = -2.5677995973755325
adf_automated_lag_cds = []; adf_automated_lag_bond = []

#create loop to test for stationarity in each GVKEY's data group 
for i, val in enumerate(sorted_alldata_df_final.groupby('gvkey')):
    gvkey = val[0]
    data = val[1]                        #assign the remainder of the data to be callable from data.(variable)
    gvkey_list.append(gvkey)             #append the GVKEY to a list, which keeps the data stored under the appropriate key
    sector_string = data.sector
    sector_list.append(sector_string.iloc[0])      #append the respective sector for that GVKEY
    
    #compute the specifics of each GVKEY's date / time horizon
    first_date_obs.append(data.trade_date.min())
    last_date_obs.append(data.trade_date.max())
    time_difference = data.trade_date.max() - data.trade_date.min()
    time_horizon.append(time_difference.days)               #append just the .days aspect of the timedelta variable
    
    #define the appropriate series for the ADF test: 
    cds = data.market_cds_spread         
    pecds = data.pecds
    vix = data.cboe_vix
    
    #the VAR model package requires the data be structured in a particular way; np.column stack is the fastest method
    data = np.column_stack((cds, pecds)) 
    
    #the VAR model construction is necessary for computing the BIC
    model = smt.VAR(data)  

    #compute the optimised BIC value from [model] which holds the vector autoregression model per GVKEY
    try:
        #for daily data, a maximum lag length is imposed (5 trading days, roughly equivalent to one trading week)
        bic_selected_lag = model.select_order(maxlags=10).bic 
        optimal_lag_list.append(bic_selected_lag)                  #store the optimised BIC result per GVKEY
        
        #ADF test - plugging in the bic selected lag length to run a unique length ADF test per GVKEY, per instrument
        test_ADF_cds = ADF(cds, trend='c', max_lags=10, method='BIC') 
        adf_automated_lag_cds.append(test_ADF_cds.lags)
        
        test_ADF_bond = ADF(pecds, trend='c',  max_lags=10, method='BIC')  
        adf_automated_lag_bond.append(test_ADF_bond.lags)
        
        test_ADF_vix = ADF(vix, trend='c',  max_lags=10, method='BIC')
        #test_ADF_vix = ADF(vix, lags=bic_selected_lag)
        
        #store the test statistic & p-value for each instrument's ADF test result, for each GVKEY (rounded to 4 decimals)
        stationarity_ts_cds.append(np.round(test_ADF_cds.stat,4))    
        stationarity_ts_bond.append(np.round(test_ADF_bond.stat,4))     
        stationarity_ts_vix.append(np.round(test_ADF_vix.stat,4))
        
        stationarity_pvalue_cds.append(np.round(test_ADF_cds.pvalue,4))    
        stationarity_pvalue_bond.append(np.round(test_ADF_bond.pvalue,4))
        stationarity_pvalue_vix.append(np.round(test_ADF_vix.pvalue,4))
        
        ##OPTIONAL
        #this part reads the test stat per GVKEY / instrument and assigns the appropriate (*) per its significance level
        #E.G. > 99% significance is commonly denoted as [test_statistic]*** in publication regression tables
        
        #CDS ADF LATEX TABLE CONSTRUCTION
        if test_ADF_cds.stat < one_percent_cv:
            stationarity_test_res_cds.append("***")   
        elif test_ADF_cds.stat < five_percent_cv:
            stationarity_test_res_cds.append("**")    ## The 1% & 5% significance would reject H0
        elif test_ADF_cds.stat < ten_percent_cv:
            stationarity_test_res_cds.append("*")     ## The 10% and no level would fail to reject H0
        else: 
            stationarity_test_res_cds.append(" ")  

        #repeat for BONDS:
        if test_ADF_bond.stat < one_percent_cv:
            stationarity_test_res_bond.append("***")
        elif test_ADF_bond.stat < five_percent_cv:
            stationarity_test_res_bond.append("**")   ## The 1% & 5% significance would reject H0
        elif test_ADF_bond.stat < ten_percent_cv:
            stationarity_test_res_bond.append("*")    ## The 10% and no level would fail to reject H0
        else: 
            stationarity_test_res_bond.append(" ")
            
            
        #VIX ADF LATEX TABLE CONSTRUCTION
        if test_ADF_vix.stat < one_percent_cv:
            stationarity_test_res_vix.append("***")   
        elif test_ADF_vix.stat < five_percent_cv:
            stationarity_test_res_vix.append("**")    ## The 1% & 5% significance would reject H0
        elif test_ADF_vix.stat < ten_percent_cv:
            stationarity_test_res_vix.append("*")     ## The 10% and no level would fail to reject H0
        else: 
            stationarity_test_res_vix.append(" ")  
                    
        
        #ensure that both series are integrated of the same order: a jointly unit-root system is required for VECM models
        if (test_ADF_bond.stat > five_percent_cv and test_ADF_cds.stat > five_percent_cv):  
            #this is > as to "not reject" the H0 is unit root; while reject the null would imply a stationary series
            stat_test_joint.append("BOND/CDS Both Have Unit Root")
            #stat_boolean.append(int(1))
        elif (test_ADF_bond.stat > five_percent_cv and test_ADF_cds.stat < five_percent_cv):
            stat_test_joint.append("BOND ONLY HAS UR")
        elif (test_ADF_bond.stat < five_percent_cv and test_ADF_cds.stat > five_percent_cv):
            stat_test_joint.append("CDS ONLY HAS UR")            
        else: 
            stat_test_joint.append("Cannot Reject H0")
            #stat_boolean.append(int(2))
        
    # A linear-algebra error could be caused by market factors which have created highly erratic basis behaviour
    # This could be caused by a firm nearing bankruptcy where its basis deviates excessively far from par
    #accordingly, the leading minor of the array is not positive definite is akin to taking the square root of a negative number
    except np.linalg.LinAlgError as lae:
        problematic_gvkey.append(gvkey)                                 #append problem GVKEY to seperate list
        print("LinAlgError: Row: {}, Reason : {} ".format(i, lae), '\n', "GVKEY: ({})".format(gvkey), 
              "SECTOR: ", sector_string.iloc[0])
      
        #append items to maintain placement in main DF \\ 
        #where the errors occur, fill with NaN values instead
        optimal_lag_list.append(np.nan)
        adf_automated_lag_cds.append(np.nan)
        adf_automated_lag_bond.append(np.nan)
        stat_test_joint.append("LinAlg_Error")
        
        stationarity_test_res_cds.append("_LAE")
        stationarity_test_res_bond.append("_LAE")
        stationarity_test_res_vix.append("_LAE")
        
        stationarity_ts_cds.append(np.nan)
        stationarity_ts_bond.append(np.nan)
        stationarity_ts_vix.append(np.nan)
        
        stationarity_pvalue_cds.append(np.nan)
        stationarity_pvalue_bond.append(np.nan)
        stationarity_pvalue_vix.append(np.nan)
        
        linalg_errors += 1
        continue
    
    #some series may have encounted some errors with a constant, thus adding a constant to the equation is invalid
    #this occurs due to poor liquidity in some markets, resulting in long periods of near-identical price quotes
    except (ValueError) as ve:
        problematic_gvkey.append(gvkey)
        print("ValueError: Row: {}, Reason : {} ".format(i, ve), '\n', "GVKEY: ({})".format(gvkey), 
              "SECTOR:", sector_string.iloc[0])
        
        #append items to maintain placement in main DF \\ 
        #where the errors occur, fill with NaN values instead
        optimal_lag_list.append(np.nan)
        adf_automated_lag_cds.append(np.nan)
        adf_automated_lag_bond.append(np.nan)        
        stat_test_joint.append("Value_Error")
        
        stationarity_test_res_cds.append("_VE")
        stationarity_test_res_bond.append("_VE")
        stationarity_test_res_vix.append("_VE")        
        
        stationarity_ts_cds.append(np.nan)
        stationarity_ts_bond.append(np.nan)
        stationarity_ts_vix.append(np.nan)
        
        stationarity_pvalue_cds.append(np.nan)
        stationarity_pvalue_bond.append(np.nan)
        stationarity_pvalue_vix.append(np.nan) 
        
        value_errors += 1
        continue  
        
#this section combines the test-statistic itself with the appropriate (*, **, ***) into one cell for the final DF
cds_sig = ["{}{}".format(stationarity_ts_cds, stationarity_test_res_cds) for stationarity_ts_cds, stationarity_test_res_cds in zip(stationarity_ts_cds, stationarity_test_res_cds)]
bond_sig = ["{}{}".format(stationarity_ts_bond, stationarity_test_res_bond) for stationarity_ts_bond, stationarity_test_res_bond in zip(stationarity_ts_bond, stationarity_test_res_bond)]  
vix_sig = ["{}{}".format(stationarity_ts_vix, stationarity_test_res_vix) for stationarity_ts_vix, stationarity_test_res_vix in zip(stationarity_ts_vix, stationarity_test_res_vix)]        

#the final DF constructor; the .T is to transpose to a more visually-friendly vertically orientated DF
df_stat_cdsbond_results = pd.DataFrame(data=(gvkey_list, sector_list, 
                                             cds_sig, adf_automated_lag_cds, stationarity_pvalue_cds,
                                             bond_sig, adf_automated_lag_bond, stationarity_pvalue_bond,
                                             vix_sig, stationarity_pvalue_vix,
                                             stat_test_joint, optimal_lag_list,
                                            first_date_obs, last_date_obs, time_horizon)).T

#dont think the boolean is needed? can read col names fine? 

#rename the columns for better readability
df_stat_cdsbond_results.columns = ['gvkey', 'sector', 
                                   'CDS', 'CDS_ADF_LAGS','CDS_pval', 
                                   'Bond', 'BOND_ADF_LAGS','BOND_pval',
                                   'VIX', 'VIX_pval',
                                   'Joint_Test_Outcome', 'bic_lag',
                                  'date_start','date_end','length_days'] 

#change to tabulate? 
print('\n', " Linear Algebra Errors #", linalg_errors, '\n', " Value Errors #", value_errors, '\n', " Total Errors #:", 
     (linalg_errors + value_errors))

In [ ]:
#max lags = 10  restrictions:
bond_lag_series = pd.to_numeric(df_stat_cdsbond_results['BOND_ADF_LAGS'], errors='coerce')
print(bond_lag_series.mean())
print(bond_lag_series.median())
print(bond_lag_series.var())
print("BOND_LAGS (>2)", np.count_nonzero(bond_lag_series >2))
print("BOND_LAGS ==(2)", np.count_nonzero(bond_lag_series ==2))
print("BOND_LAGS ==(1)", np.count_nonzero(bond_lag_series ==1))
print("BOND_LAGS ==(0)", np.count_nonzero(bond_lag_series ==0))

#with max lags == 10 restrictions
print('\n', "CDS")
cds_lag_series = pd.to_numeric(df_stat_cdsbond_results['CDS_ADF_LAGS'], errors='coerce')
print(cds_lag_series.mean())
print(cds_lag_series.median())
print(cds_lag_series.var())
print("CDS_LAGS (>2)", np.count_nonzero(cds_lag_series >2))
print("CDS_LAGS ==(2)", np.count_nonzero(cds_lag_series ==2))
print("CDS_LAGS ==(1)", np.count_nonzero(cds_lag_series ==1))
print("CDS_LAGS ==(0)", np.count_nonzero(cds_lag_series ==0))